Acknowledgement:
    1. The structure of the code is inspired by depmixS4: An R Package for Hidden Markov Models: https://cran.r-project.org/web/packages/depmixS4/vignettes/depmixS4.pdf
    2. Some of the linear model codes are adapted from sklearn: http://scikit-learn.org/stable/ and statsmodel: http://statsmodels.sourceforge.net/. Some modifications have been made to these codes to realize more functionalities.

Problems with existing packages
    1. Some of sklearn and statsmodels does not support the implementation of sample weights
    2. Some of sklearn and statsmodels does not support l1, l2 or elasticnet regularizations
    3. Sklearn packages does not support estimation of standard deviation of coefficients
    4. The likelihood function of weighted linear models is not the same as the ones we need to use in IO-HMM
    5. In the R package aformentioned, they do not support the provision of multiple sequences.

Modifications to above packages:
    1. Implemented supervised models that supports sample weights
    2. Supports the estimation of standard deviations of coefficients
    3. Supports multiple regularizations (l1, l2, elastic net) in most of the supervised models. (However,  if regularization is applied, no standard deviation of the coefficients will be estimated)
    4. Supports estimation over multiple sequences (multiple dataframes)
    5. HMM forward-backward code was implemented at the log scale so that it is more robust to long sequences.
    6. Supports generalized linear models with different link functions, just as statsmodel.

# Example use of UnSupervised_IOHMM

In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import  division

import warnings


import numpy as np
import pandas as pd


from IOHMM import UnSupervisedIOHMM
from IOHMM import UnivariateOLS, DiscreteMNL, CrossEntropyMNL


warnings.simplefilter("ignore")

## Speed data - example 1

In [2]:
speed = pd.read_csv('../data/speed.csv')
speed.head()

,Unnamed: 0,rt,corr,Pacc,prev
0,1,6.456770,cor,0.0,inc
1,2,5.602119,cor,0.0,cor
2,3,6.253829,inc,0.0,cor
3,4,5.451038,inc,0.0,inc
4,5,5.872118,inc,0.0,inc


## Setting up the model

In [3]:
SHMM = UnSupervisedIOHMM(num_states=2, max_EM_iter=200, EM_tol=1e-6)
SHMM.set_models(model_emissions = [UnivariateOLS()], 
                model_transition=CrossEntropyMNL(reg_method='l2', solver='lbfgs'),
                model_initial=CrossEntropyMNL(reg_method='l2', solver='lbfgs'))
SHMM.set_inputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.set_outputs([['rt']])
SHMM.set_data([speed])

In [4]:
SHMM.train()

## See the coefficients

In [5]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef

[ 5.51036534]
[ 6.38505427]


In [6]:
print np.sqrt(SHMM.model_emissions[0][0].dispersion)
print np.sqrt(SHMM.model_emissions[1][0].dispersion)

0.191753834053
0.24415881604


In [7]:
print np.exp(SHMM.model_transition[0].predict_log_proba(np.array([[]])))
print np.exp(SHMM.model_transition[1].predict_log_proba(np.array([[]])))

[[ 0.88351271  0.11648729]]
[[ 0.08433109  0.91566891]]


## Speed data - example 2

In [8]:
SHMM = UnSupervisedIOHMM(num_states=2, max_EM_iter=200, EM_tol=1e-6)
SHMM.set_models(model_emissions = [UnivariateOLS(est_stderr=True), 
                                   DiscreteMNL(reg_method='l2', solver='lbfgs')], 
                model_transition=CrossEntropyMNL(reg_method='l2', solver='lbfgs'),
                model_initial=CrossEntropyMNL(reg_method='l2', solver='lbfgs'))
SHMM.set_inputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[],['Pacc']])
SHMM.set_outputs([['rt'],['corr']])
SHMM.set_data([speed])

In [9]:
SHMM.train()

In [10]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[0][1].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[1][1].coef

[ 6.38764086]
[[-1.13692639 -2.17390907]]
[ 5.51390441]
[[-0.21848254  0.57762286]]


In [11]:
print SHMM.model_emissions[0][0].stderr
print SHMM.model_emissions[0][1].stderr
print SHMM.model_emissions[1][0].stderr
print SHMM.model_emissions[1][1].stderr

[ 0.01498921]
None
[ 0.01433147]
None


In [12]:
print np.exp(SHMM.model_transition[0].predict_log_proba(np.array([[]])))
print np.exp(SHMM.model_transition[1].predict_log_proba(np.array([[]])))

[[ 0.91491045  0.08508955]]
[[ 0.11590526  0.88409474]]
